# A Guided Tour of Ray Core: Remote Objects

© 2019-2022, Anyscale. All Rights Reserved

[*Remote Objects*](https://docs.ray.io/en/latest/walkthrough.html#objects-in-ray)
implement a [*shared-memory object store*](https://en.wikipedia.org/wiki/Shared_memory) pattern.

Objects are immutable, and can be accessed from anywhere on the cluster, as they are stored in the cluster shared memory.

<img src="../images/shared_memory.png"  height="40%" width="65%">

In general, small objects are stored in their owner’s **in-process store** while large objects are stored in the **distributed object store**. This decision is meant to reduce the memory footprint and resolution time for each object. Note that in the latter case, a placeholder object is stored in the in-process store to indicate the object has been promoted to shared memory.

In the case if there is no space in the shared-memory, objects are spilled over to disk. But the main point here is that
shared-memory allows zero-copy to processes on the same worker node.

<img src="../images/shared_memory_plasma_store.png"  height="40%" width="65%">

---

## 2. Object references as Futures Pattern

First, let's start Ray…

In [4]:
import logging
import ray

ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

## Remote Objects example

To start, we'll define a remote object...

In [5]:
%%time

num_list = [23, 42, 93]

# returns an objectRef
obj_ref = ray.put(num_list)
obj_ref

CPU times: user 870 µs, sys: 753 µs, total: 1.62 ms
Wall time: 823 µs


ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000002000000)

Then retrieve the value of this object reference. This follows an object resolution protocol.

<img src="../images/object_resolution.png" height="40%" width="65%">

Small objects are resolved by copying them directly from the owner’s **in-process store**. For example, if the owner calls `ray.get`, the system looks up and deserializes the value from the local **in-process store**. If the owner submits a dependent task, it inlines the object by copying the value directly into the task description. Note that these objects are local to the owner process: if a borrower attempts to resolve the value, the object is promoted to shared memory, where it can be retrieved through the distributed object resolution protocol described next.

Resolving a large object. The object `x` is initially created on Node 2, e.g., because the task that returned the value ran on that node. This shows the steps when the owner (the caller of the task) calls `ray.get`: 

 1) Lookup object’s locations at the owner. 
 2) Select a location and send a request for a copy of the object. 
 3) Receive the object.



In [6]:
%%time

val = ray.get(obj_ref)
val

CPU times: user 899 µs, sys: 787 µs, total: 1.69 ms
Wall time: 878 µs


[23, 42, 93]

Let's combine use of a remote function with a remote object, to illustrate *composable futures*:

In [7]:
@ray.remote
def my_function (num_list):
    return sum(num_list)

In other words, the remote function `myfunction()` will sum the list of integers in the remote object `num_list`:

In [8]:
%%time

calc_ref = my_function.remote(obj_ref)

CPU times: user 3.77 ms, sys: 2.01 ms, total: 5.79 ms
Wall time: 5.29 ms


In [9]:
%%time

result = ray.get(calc_ref)
result

CPU times: user 408 µs, sys: 115 µs, total: 523 µs
Wall time: 490 µs


158

You can gather the values of multiple object references in parallel using collections:
 1. Each value is put in the object store and its objRef is immediately returned
 2. The comprehsion constructs a list of objRefs for each element in the loop
 3. A final `get(list_obj_refs`) is invoked to fetch the list

In [14]:
%%time

results = ray.get([ray.put(i) for i in range(10)])
results

CPU times: user 6.53 ms, sys: 4.88 ms, total: 11.4 ms
Wall time: 5.7 ms


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### What about long running tasks?

Now let's set a timeout to return early from attempted access of a remote object that is blocking for too long...

In [19]:
import time

@ray.remote
def long_running_function ():
    time.sleep(10)
    return 42

You can control how long you want to wait for the task to finish

In [20]:
%%time

from ray.exceptions import GetTimeoutError

obj_ref = long_running_function.remote()

try:
    ray.get(obj_ref, timeout=6)
except GetTimeoutError:
    print("`get` timed out")

`get` timed out
CPU times: user 30.8 ms, sys: 26.5 ms, total: 57.3 ms
Wall time: 6.02 s


In [22]:
ray.shutdown()

### Excercise

1. Send a list of object references returned by `ray.put(x)` 
2. Use comprehension to construct this list and send it `my_function.remote(list_of_object_refs)` to return the sum of the list

### Solutions

1.`ray.get(my_function.remote(ray.get([ray.put(i) for i in range(10)])))`

## References

 * [Ray Architecture Reference](https://docs.google.com/document/d/1lAy0Owi-vPz2jEqBSaHNQcy2IBSDEHyXNOQZlGuj93c/preview#)

 * [Ray Internals: A peek at ray,get](https://www.youtube.com/watch?v=a1kNnQu6vGw)

 * [Ray Internals: Object management with Ownership Model](https://www.anyscale.com/events/2021/06/22/ray-internals-object-management-with-the-ownership-model)

 * [Deep Dive into Ray scheduling Policies](https://www.anyscale.com/events/2021/06/23/a-deep-dive-into-rays-scheduling-policy)